In [12]:
from urllib2 import urlopen
from urllib2 import HTTPError
from bs4 import BeautifulSoup
import pandas as pd
import re

In [13]:
def getFilmTable(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        print("err http")
        return e
    try:
        soup = BeautifulSoup(html,"lxml")
        return soup.find_all("table",class_="wikitable sortable")[0]
    except AttributeError as e:
        print("err atr")
        return e

In [14]:
film_table = getFilmTable('https://en.wikipedia.org/wiki/Ryan_Gosling')

In [15]:
Year=[]
Title=[]
Role=[]

header_row = True
for row in film_table.findAll("tr"):
    if not header_row:
        cells = row.findAll('td')
        index_of_title = 1
        if len(cells) == 3:
            index_of_title = 0
            last_inserted_year = Year[-1]
            Year.append(last_inserted_year)
        else:
            Year.append(cells[0].find(text=True))
        
        Title.append(cells[index_of_title])
        role = cells[index_of_title + 1].find(text=True)
        if role == None:
            Role.append(cells[index_of_title + 2].find(text=True))
        else:
            Role.append(role)
    header_row = False

In [223]:
df=pd.DataFrame(Year,columns=['Date'])
df['Title']=list(map(lambda title : title.find(text=True), Title))
df['Role']=Role
df

,Date,Title,Role
0,1997,Frankenstein and Me,Kenny
1,2000,Remember the Titans,Alan Bosley
2,2001,"Believer, The",Danny Balint
3,2002,Murder by Numbers,Richard Haywood
4,2002,"Slaughter Rule, The",Roy Chutney
5,2003,"United States of Leland, The",Leland P. Fitzgerald
6,2004,The Notebook,Noah Calhoun
7,2005,Stay,Henry Letham
8,2006,Half Nelson,Dan Dunne
9,2007,Fracture,Willy Beachum


In [467]:
films_with_coactor = []
coactor = []
coactor_href = []
year_of_birth = []
country = []
awards = []

In [468]:
def get_actor_awards_from_url(url):
    url = 'https://en.wikipedia.org' + url
    try:
        html = urlopen(url)
    except HTTPError as e:
        print("err http")
        return e
    try:
        soup = BeautifulSoup(html,"lxml")
        number_of_awards = 0
        table = soup.find_all("table",class_ = 'infobox')
        try:
            def checker(yes):
                if yes.text == "Won":
                    return 1
                else:
                    return 0
            number_of_awards = reduce(lambda acc,yes: acc + checker(yes),[0] + soup.find_all(class_="yes"))
            if number_of_awards == 0:
                if(len(table[0].find_all('dt',text="Total Awards"))):
                    wins = int(table[0].find_all("td",class_="yes")[-1].text)
                    number_of_awards = wins
                else:
                    wins = table[0].find_all("td",class_="yes")
                    number_of_awards = reduce(lambda acc,td: int(td.text)+acc, [0] + wins)
        except:
            number_of_awards = "Format unexpected for hw1"
        return number_of_awards
    except AttributeError as e:
        print("err atr")
        return e

In [469]:
def get_actor_awards(actor_name):
    print(actor_name)
    last_name = actor_name.split(" ")[-1]
    try:
        url = "https://en.wikipedia.org/w/index.php?title=Category:Lists_of_awards_by_actor&from=" + last_name[0]
        html = urlopen(url)
    except HTTPError as e:
        print("err http")
        return e
    try:
        soup = BeautifulSoup(html,"lxml")
        number_of_awards = "Not avlible"
        link = soup.find_all("a",text=re.compile(r"(" + actor_name + r")"))
        if len(link) != 0:
            number_of_awards = get_actor_awards_from_url(link[0]['href'])
        print(number_of_awards)
        return number_of_awards
    except AttributeError as e:
        print("err atr")
        return e


In [470]:
def get_actor_info(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        print("err http")
        return e
    try:
        soup = BeautifulSoup(html,"lxml")
        name = soup.find("h1",id="firstHeading").find(text=True)
        try:
            actor_info = filter(lambda table : len(table.find_all('th',text="Born")),soup.find_all('tr'))[0]
            year_of_birth = actor_info.find_all('span',class_='bday')[0].text[0:4]
            full_address =actor_info.find_all('span',class_='birthplace')[0].text
            country = full_address[full_address.rfind(",")+1:]
        except:
            try:
                year_text = soup.find_all(text=re.compile(r"\(([^(])*born([^)]+)\)"))[0]
                year_text = year_text[year_text.find(")")-4,year_text.find(")")]
                year_of_birth = year_text
            except:
                year_of_birth = "Not avlible"
            

            country = "Not avlible"
        number_of_awards = get_actor_awards(name)
        return name,year_of_birth,country,number_of_awards
    except AttributeError as e:
        print("err atr")
        return e

In [471]:
def get_film_and_cast_href(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        print("err http")
        return e
    try:
        soup = BeautifulSoup(html,"lxml")
        title = soup.find("h1",id="firstHeading").find_all('i')[0].find(text=True)
        try:
            cast_section = filter(lambda table : len(table.find_all('th',text="Starring")),soup.find_all('tr'))
            actors_with_href = filter(lambda actor_li : len(actor_li.find_all('a')) != 0 ,cast_section[0].find_all('li'))
            actors_hrefs_and_titles = map(lambda actor_li : (actor_li.text , actor_li.find_all('a')[0]['href'] ),actors_with_href)
            if len(actors_with_href) == 0:
                actors_with_href = filter(lambda actor_td : len(actor_td.find_all('a')) != 0 ,cast_section[0].find_all('td'))
                actors_hrefs_and_titles = map(lambda actor_a : (actor_a.text , actor_a['href'] ),actors_with_href[0].find_all('a'))

            actors_info_without_mined_actor = filter(lambda info : info[1].lower() != 'ryan gosling',actors_hrefs_and_titles)
            actors_hrefs = map(lambda info : 'https://en.wikipedia.org' + info[1],actors_info_without_mined_actor)
        except:
            actors_hrefs = []
        return actors_hrefs,title
    except AttributeError as e:
        print("err atr")
        return e

In [472]:
def compose_film_fection(url):
    print url
    cast_href , title = get_film_and_cast_href(url)
    for href in cast_href:
        try:
            index_of_actor = coactor_href.index(href)
        except ValueError as e:
            index_of_actor = -1
            
        if index_of_actor != -1:
            films_with_coactor[index_of_actor].append(title)
        else:
            coactor_href.append(href)
            films_with_coactor.append([title])
            name,year_of_birth_actor,country_of_birth,number_of_awards = get_actor_info(href)
            coactor.append(name)
            year_of_birth.append(year_of_birth_actor)
            country.append(country_of_birth)
            awards.append(number_of_awards)
            
    return title
    

In [473]:
movies_hrefs = map(lambda title_a : 'https://en.wikipedia.org' + title_a[0]['href'] ,filter(lambda x : x != [],list(map(lambda title : title.find_all('a') , Title))))
    

In [475]:
map(lambda movie : compose_film_fection(movie),movies_hrefs)

https://en.wikipedia.org/wiki/Remember_the_Titans
https://en.wikipedia.org/wiki/The_Believer_(film)
https://en.wikipedia.org/wiki/Murder_by_Numbers
https://en.wikipedia.org/wiki/The_Slaughter_Rule
https://en.wikipedia.org/wiki/The_United_States_of_Leland
https://en.wikipedia.org/wiki/The_Notebook
https://en.wikipedia.org/wiki/Stay_(2005_film)
https://en.wikipedia.org/wiki/Half_Nelson_(film)
https://en.wikipedia.org/wiki/Fracture_(2007_film)
https://en.wikipedia.org/wiki/Lars_and_the_Real_Girl
https://en.wikipedia.org/wiki/Blue_Valentine_(film)
https://en.wikipedia.org/wiki/All_Good_Things_(film)
https://en.wikipedia.org/wiki/Crazy,_Stupid,_Love
https://en.wikipedia.org/wiki/Drive_(2011_film)
https://en.wikipedia.org/wiki/The_Ides_of_March_(film)
https://en.wikipedia.org/wiki/The_Place_Beyond_the_Pines
https://en.wikipedia.org/wiki/Gangster_Squad_(film)
https://en.wikipedia.org/wiki/Only_God_Forgives
https://en.wikipedia.org/wiki/White_Shadow_(film)
https://en.wikipedia.org/wiki/Lost_Ri

[u'Remember the Titans',
 u'The Believer',
 u'Murder by Numbers',
 u'The Slaughter Rule',
 u'The United States of Leland',
 u'The Notebook',
 u'Stay',
 u'Half Nelson',
 u'Fracture',
 u'Lars and the Real Girl',
 u'Blue Valentine',
 u'All Good Things',
 u'Crazy, Stupid, Love',
 u'Drive',
 u'The Ides of March',
 u'The Place Beyond the Pines',
 u'Gangster Squad',
 u'Only God Forgives',
 u'White Shadow',
 u'Lost River',
 u'The Big Short',
 u'The Nice Guys',
 u'La La Land',
 u'Song to Song',
 u'Blade Runner 2049',
 u'First Man']

In [476]:
df2=pd.DataFrame(coactor,columns=['Name'])
df2['Movies']=films_with_coactor
df2['Year of birth'] = year_of_birth
df2['Country of birth'] = country
df2['num. of awards'] = awards
df2

,Name,Movies,Year of birth,Country of birth,num. of awards
0,Denzel Washington,"[Remember the Titans, Remember the Titans]",1954,U.S.,60
1,Will Patton,"[Remember the Titans, Remember the Titans]",1954,U.S.,Not avlible
2,Donald Faison,"[Remember the Titans, Remember the Titans]",1974,U.S.,Not avlible
3,Nicole Ari Parker,"[Remember the Titans, Remember the Titans]",1970,U.S.,Not avlible
4,Ryan Gosling,"[The Believer, Murder by Numbers, The Slaughte...",1980,Canada,21
5,Billy Zane,"[The Believer, The Believer]",1966,U.S.,Not avlible
6,Theresa Russell,"[The Believer, The Believer]",1957,U.S.,Not avlible
7,Summer Phoenix,"[The Believer, The Believer]",1978,U.S.,Not avlible
8,Sandra Bullock,"[Murder by Numbers, Murder by Numbers]",1964,U.S.,101
9,Ben Chaplin,"[Murder by Numbers, Murder by Numbers]",1969,England,Not avlible
